In [1]:
# Demo code of hyperboloid GP-LVM
# Our source code does not require any GPU dependency.
# Our code depends on numpy, scipy, and tqdm.

# python:3.10.12, numpy: 1.26.2, scipy: 1.11.4, tqdm: 4.66.1

In [1]:

# import library and modules

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os

from hyperboloid_gplvm import HyperboloidGPLVM
from sparse_hyperboloid_gplvm import SparseHyperboloidGPLVM
from bayesian_hyperboloid_gplvm import BayesianHyperboloidGPLVM


In [2]:

# function for scatter plot on the Poincare ball
# We use the code presented in the PoincareMap implementation (https://www.nature.com/articles/s41467-020-16822-4)
# We appreciate their contribution.

def linear_scale(embeddings):
    embeddings = np.transpose(embeddings)
    sqnorm = np.sum(embeddings ** 2, axis=1, keepdims=True)
    dist = np.arccosh(1 + 2 * sqnorm / (1 - sqnorm))
    dist = np.sqrt(dist)
    dist /= dist.max()
    sqnorm[sqnorm == 0] = 1
    embeddings = dist * embeddings / np.sqrt(sqnorm)
    return np.transpose(embeddings)

def plotPoincareDisc(x,
                     label_names=None,
                     file_name=None,
                     title_name=None,
                     idx_zoom=None,
                     show=False,
                     d1=12,
                     d2=6,
                     fs=11,
                     ms=4,
                     col_palette=None,
                     color_dict=None,
                     legend=True):
    if col_palette is None:
        # col_palette = colors_palette
        # col_palette = plt.get_cmap("tab10")
        col_palette = sns.color_palette('deep')

    df = pd.DataFrame(dict(x=x[0], y=x[1], label=label_names))
    groups = df.groupby('label')

    fig = plt.figure(figsize=(d1, d2), dpi=300)
    circle = plt.Circle((0, 0), radius=1,  fc='none', color='black')

    plt.subplot(1, 2, 1)
    plt.gca().add_patch(circle)
    plt.plot(0, 0, 'x', c=(0, 0, 0), ms=ms)
    plt.title(title_name, fontsize=fs)

    if color_dict is None:
        j = 0
        color_dict = {}
        for name, group in groups:
            color_dict[name] = col_palette[j]
            j += 1

    marker = '.'
    for name, group in groups:        
        plt.plot(group.x, group.y, marker=marker, markerfacecolor='none',
                 c=color_dict[name], linestyle='', ms=ms, label=name)
    plt.plot(0, 0, 'x', c=(0, 0, 0), ms=ms)
    plt.axis('off')
    plt.axis('equal')
    # plt.legend(numpoints=1, loc='center left',
    #            bbox_to_anchor=(1, 0.5), fontsize=fs)

    labels_list = np.unique(label_names)
#
    if idx_zoom is None:
        xl = np.array(linear_scale(x))
        xl[np.isnan(xl)] = 0

        df = pd.DataFrame(dict(x=xl[0], y=xl[1], label=label_names))
        groups = df.groupby('label')
    else:
        xl = np.array(linear_scale(x[:, idx_zoom]))
        xl[np.isnan(xl)] = 0

        df = pd.DataFrame(dict(x=xl[0], y=xl[1], label=label_names[idx_zoom]))
        groups = df.groupby('label')

    circle = plt.Circle((0, 0), radius=1, fc='none',
                        color='black', linestyle=':')
    plt.subplot(1, 2, 2)
    plt.gca().add_patch(circle)
    # plt.plot(0, 0, 'x', c=(0, 0, 0), ms=ms)
    plt.title('zoom in', fontsize=fs)
    marker = 'o'
    for name, group in groups:
        plt.plot(group.x, group.y, marker=marker, markerfacecolor='none',
                 c=color_dict[name], linestyle='', ms=ms, label=name)

    plt.plot(0, 0, 'x', c=(0, 0, 0), ms=ms)

    plt.axis('off')
    plt.axis('equal')

#     for l in labels_list:
# #         i = np.random.choice(np.where(labels == l)[0])
#         ix_l = np.where(label_names == l)[0]
#         c1 = np.median(xl[0, ix_l])
#         c2 = np.median(xl[1, ix_l])
#         plt.text(c1, c2, l, fontsize=fs)
    if legend:
        plt.legend(numpoints=1, loc='center left',
               bbox_to_anchor=(1, 0.5), fontsize=fs)

    plt.tight_layout()

    if file_name:
        plt.savefig(file_name + '.svg', format='svg')

    if show:
        plt.show()

    plt.close(fig)

def diffeomorphism2ball(X):
    """
    diffeomorphism from hyperboloid to Poincare ball
    """
    return X[:,1:] / (X[:,0][:,None] + 1.)

def return_embedding(model, model_name):
    if model_name == 'hgplvm' or model_name == 'sparse_hgplvm':
        return diffeomorphism2ball(model.X)
    else:
        return diffeomorphism2ball(model.M)

def return_raw_embedding(model, model_name):
    if model_name == 'hgplvm' or model_name == 'sparse_hgplvm':
        return diffeomorphism2ball(model.X)
    else:
        return diffeomorphism2ball(model.M)


In [3]:

# function for model and data load

def return_model(model, data, lengthscale, num_inducing=None, sampling_num=None):
    if model == 'hgplvm':
        return HyperboloidGPLVM(Y=data, latent_dim=2, lengthscale=lengthscale)
    elif model == 'sparse_hgplvm':
        return SparseHyperboloidGPLVM(Y=data, latent_dim=2, lengthscale=lengthscale, num_inducing=num_inducing)
    elif model == 'bayesian_hgplvm':
        return BayesianHyperboloidGPLVM(Y=data, latent_dim=2, lengthscale=lengthscale, num_inducing=num_inducing, sampling_num=sampling_num)

def load_data(data_name):
    data = np.load('dataset/%s/train/data.npy'%(data_name))
    label = np.load('dataset/%s/train/label.npy'%(data_name), allow_pickle=True).reshape(-1)[:]
    is_file = os.path.isfile('dataset/%s/train/label_name.npy'%(data_name))
    if is_file:
        label = np.load('dataset/%s/train/label_name.npy'%(data_name), allow_pickle=True).reshape(-1)[:]
    return data, label


In [9]:

# Select dataset from ['MyeloidProgenitors', 'Paul'(scRNA-seq), 'synthetic_bin_tree/4','synthetic_bin_tree/5','synthetic_bin_tree/6']
# ('synthetic_bin_tree/4' means synthetic binary tree dataset with depth 4.)
data_name = 'synthetic_bin_tree/5'

# Color code for Paul dataset (from PoincareMap implementation)
color_dict_paul = {'12Baso': '#0570b0', '13Baso': '#034e7b',
	 '11DC': '#ffff33', 
	 '18Eos': '#2CA02C', 
	 '1Ery': '#fed976', '2Ery': '#feb24c', '3Ery': '#fd8d3c', '4Ery': '#fc4e2a', '5Ery': '#e31a1c', '6Ery': '#b10026',
	 '9GMP': '#999999', '10GMP': '#4d4d4d',
	 '19Lymph': '#35978f', 
	 '7MEP': '#E377C2', 
	 '8Mk': '#BCBD22', 
	 '14Mo': '#4eb3d3', '15Mo': '#7bccc4',
	 '16Neu': '#6a51a3','17Neu': '#3f007d',
	 'root': '#000000'}

# Define model and hyperparameter
# We recommend hgplvm for synthetic data and Bayesian hgplbm for scRNA-seq data (but time consuming)

# select model ['hgplvm', 'sparse_hgplvm', 'bayesian_hgplvm']
# Bayesian hGP-LVM is slow due to the sampling approximation. 
model_name = 'hgplvm'

# set lengthscale (we recommend 'lengthscale=100')
lengthscale=100

# set num_inducing M (sparse and Bayesian hGP-LVM) and number of the sampling (Bayesian hGP-LVM)
num_inducing = 50
sampling_num = 5


In [ ]:

# Learning hGP-LVMs
data, label = load_data(data_name)
model = return_model(model_name, data, lengthscale, num_inducing, sampling_num)

color_dict = None
if data_name=='Paul':
    color_dict = color_dict_paul
if 'synthesis_bin_tree' in data_name:
    y_set = set(label)
    colors_palette = sns.color_palette("deep",len(y_set))

# We recommend 'max_iters=1500' for Bayesian hGP-LVM on Paul dataset
model.RiemannGD(max_iters=1500)
embedding = return_embedding(model, model_name)
title = model_name + '_' + 'lengthscale_' + str(lengthscale)

plotPoincareDisc(embedding.T, label, file_name=title, title_name=title, show=True, color_dict=color_dict, legend=False)


  0%|          | 0/1500 [00:00<?, ?it/s]

0-th iter: mean norm:0.01858, obj:18265.11566
10-th iter: mean norm:0.25697, obj:17240.53957
20-th iter: mean norm:0.22452, obj:16120.48083
30-th iter: mean norm:0.21768, obj:14857.12891
40-th iter: mean norm:0.21135, obj:13404.62709
50-th iter: mean norm:0.21666, obj:11696.58300
60-th iter: mean norm:0.21958, obj:9620.00956


In [6]:
# quantitative evaluation for bin_tree data

from hyperboloid import Hyperboloid

# binary representation of data
true_data = np.load('dataset/%s/train/true_data.npy'%(data_name))

# compute hamming distance
# We use the implementation presented in 
# https://proceedings.neurips.cc/paper_files/paper/2022/file/71ad539a57b1fd49b19e5c80070cb8b9-Paper-Conference.pdf 

d_true = np.zeros((true_data.shape[0],true_data.shape[0]))
for i in range(d_true.shape[0]):
    for j in range(d_true.shape[1]):
        d_true[i][j] = (true_data[i].astype(np.int32) ^ true_data[j].astype(np.int32)).sum()

comp = Hyperboloid(2)
d_embedding = comp.distance(model.X)
mask = np.fromfunction(lambda i, j: i > j, shape=d_true.shape)
corr_distance = np.corrcoef(d_embedding[mask].ravel(), d_true[mask].ravel())[0, 1]

print(corr_distance)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/MyeloidProgenitors/train/true_data.npy'